In [1]:
# https://drive.google.com/file/d/1qAgLKmiIJr3YGklhSHv277_QBFOhdnJP/view?usp=sharing
# !wget wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qAgLKmiIJr3YGklhSHv277_QBFOhdnJP' -O tonedataset.zip
# !gdown 1qAgLKmiIJr3YGklhSHv277_QBFOhdnJP
# https://drive.google.com/file/d/1hfvaRHlNDrx-wIB5vEjuicDOZ0UC7LX2/view?usp=drivesdk
!gdown 1hfvaRHlNDrx-wIB5vEjuicDOZ0UC7LX2

Downloading...
From: https://drive.google.com/uc?id=1hfvaRHlNDrx-wIB5vEjuicDOZ0UC7LX2
To: /content/cropped_dataset.zip
100% 25.1M/25.1M [00:00<00:00, 65.0MB/s]


In [2]:
import zipfile

local_zip = './cropped_dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./cropped_dataset')
zip_ref.close()

In [48]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  """
  Creates the training and validation data generators

  Args:
    TRAINING_DIR (string): directory path containing the training images
    VALIDATION_DIR (string): directory path containing the testing/validation images

  Returns:
    train_generator, validation_generator - tuple containing the generators
  """

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  train_datagen = ImageDataGenerator( rescale = 1.0/255.,
                                    #  rotation_range = 40,
                                    #  width_shift_range = 0.2,
                                    #  height_shift_range = 0.2,
                                     shear_range = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     fill_mode = 'nearest' )

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=64,color_mode='rgb',
                                                      class_mode='categorical',
                                                      target_size=(576, 768))

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator( rescale = 1.0/255. )

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=64,
                                                                class_mode='categorical',
                                                                target_size=(576, 768))
  return train_generator, validation_generator

In [49]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 863 images belonging to 10 classes.
Found 214 images belonging to 10 classes.


In [50]:
from tensorflow.keras.applications import MobileNetV2

In [36]:
class callBackFunc(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training after reaching 60 percent accuracy

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check accuracy
    if(logs.get('loss') < 0.2):

      # Stop if threshold is met
      print("\nLoss is lower than 0.4 so cancelling training!")
      self.model.stop_training = True

# Instantiate class
callbacks = callBackFunc()

In [37]:
base_model = MobileNetV2(input_shape=(576, 768, 3), include_top=False, weights='imagenet')
base_model.trainable = False

In [51]:
model =  tf.keras.models.Sequential([
    base_model,
     tf.keras.layers.GlobalAveragePooling2D(),
     tf.keras.layers.Dense(128, activation='relu'),
     tf.keras.layers.Dropout(0.2),
     tf.keras.layers.Dense(10, activation='softmax')
])

In [39]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [40]:
history = model.fit(train_generator,
                    epochs=50,
                    verbose=1,
                    validation_data=validation_generator,callbacks=[callbacks])

Epoch 1/50
14/14 [==============================] - 92s 6s/step - loss: 2.3804 - accuracy: 0.2074 - val_loss: 2.0495 - val_accuracy: 0.3131
Epoch 2/50
14/14 [==============================] - 88s 7s/step - loss: 1.9515 - accuracy: 0.3244 - val_loss: 1.8928 - val_accuracy: 0.3505
Epoch 3/50
14/14 [==============================] - 88s 6s/step - loss: 1.8161 - accuracy: 0.3812 - val_loss: 1.6818 - val_accuracy: 0.3925
Epoch 4/50
14/14 [==============================] - 90s 6s/step - loss: 1.6879 - accuracy: 0.4229 - val_loss: 1.8188 - val_accuracy: 0.3224
Epoch 5/50
14/14 [==============================] - 90s 6s/step - loss: 1.5479 - accuracy: 0.4542 - val_loss: 1.5264 - val_accuracy: 0.4953
Epoch 6/50
14/14 [==============================] - 90s 6s/step - loss: 1.4877 - accuracy: 0.4786 - val_loss: 1.4133 - val_accuracy: 0.5701
Epoch 7/50
14/14 [==============================] - 89s 6s/step - loss: 1.3595 - accuracy: 0.5319 - val_loss: 1.5363 - val_accuracy: 0.4019
Epoch 8/50
14/14 [==

In [46]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  # path = '/content/' + fn
  # img = load_img(path, target_size=(576, 768))
  # x = img_to_array(img)
  # x /= 255
  # x = np.expand_dims(x, axis=0)
  path = '/content/' + fn
  images = tf.io.read_file(path)
  images = tf.image.decode_png (images, channels=3)
  images = tf.image.convert_image_dtype(images, dtype=tf.float32)
  images = tf.image.resize(images, [576,768])
  images = tf.expand_dims(images, axis=0)

  label = ['Tone 1', 'Tone 2', 'Tone 3', 'Tone 4', 'Tone 5', 'Tone 6', 'Tone 7', 'Tone 8', 'Tone 9', 'Tone 10']

  # images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  prediction = np.argmax(classes[0])
  print(prediction)
  print('result is ' + label[prediction])

  print(classes)


  # if classes[0]>0.5:
  #   print(fn + " is a tone2")
  # else:
  #   print(fn + " is a tone1")

Saving obama2.jpg to obama2 (1).jpg
1/1 [==============================] - 0s 25ms/step
7
result is Tone 8
[[1.2957105e-01 2.0160722e-03 6.7891307e-02 4.4434582e-04 1.3979788e-04
  2.0826784e-01 1.8762341e-02 5.6379962e-01 1.8800910e-03 7.2275857e-03]]


In [47]:
model.save("tone_model_mobilenet89-128unit.h5")

In [ ]:
pip install pyyaml h5py

In [ ]:
import os

new_model = tf.keras.models.load_model('tone_model.h5')


ValueError: The channel dimension of the inputs should be defined. The input_shape received is (None, None, None, None), where axis -1 (0-based) is the channel dimension, which found to be `None`.

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():

  mapping = {
      'tone 1':0,
      'tone 2':1,
      'tone 3':2,
      'tone 4':3,
      'tone 5':4,
      'tone 6':5,
      'tone 7':6,
      'tone 8':7,
      'tone 9':8,
      'tone 10':9,
  }

  # predicting images
  path = '/content/' + fn
  image = tf.io.read_file(path)
  image = tf.image.decode_png (image, channels=3)
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  image = tf.image.resize(image, [576,768])
  image = tf.expand_dims(image, axis=0)
  # (60,60,3) tf.expand_dims(image, axis=0) # (1,60,60,3) image image = = image = image image =
  # img = load_img(path, target_size=(576, 768))
  # x = img_to_array(img)
  # x /= 255
  # x = np.expand_dims(x, axis=0)

  # images = np.vstack([x])

  classes = model.predict(image, batch_size=10)
  print(classes[0])
  prediction = np.argmax(classes[0])
  print(prediction)
  # print(list(mapping.values()).index(prediction))

  # Tone 1 = 0
  # Tone 2 = 2
  # Tone 10 = 1

Saving PXL_20220922_140610185.PORTRAIT.jpg to PXL_20220922_140610185.PORTRAIT.jpg
1/1 [==============================] - 0s 21ms/step
[1.0208844e-02 1.1793089e-10 9.8979038e-01 1.4588500e-08 6.4206662e-10
 9.2173282e-09 3.2264086e-10 3.4358017e-08 1.0455163e-12 8.1598728e-07]
2
